In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import re
import json
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
driver = webdriver.Chrome(executable_path="C:/Users/Yoon/Downloads/chromedriver.exe")
df = pd.read_csv("C:/Users/Yoon/Desktop/capstone/csv/카카오/강남구_한식.csv")

<ipython-input-2-3845d7aa376c>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/Users/Yoon/Downloads/chromedriver.exe")


In [3]:
#리뷰 추출 함수

def extractReview(rev_list):
    global driver
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    #리뷰 목록 찾기
    reviewList = soup.select('.list_evaluation >li')
    
    #리뷰가 있는 경우
    if len(reviewList) != 0:
        for i, review in enumerate(reviewList):
            comment = review.select('.txt_comment > span') # 리뷰
            val = ''
            if len(comment) != 0:
                if comment[0].text == '':
                    continue
                else:
                    val = comment[0].text + '\t10'
            print(val)
            rev_list.append(val)
    else:
        print('no review in extract')

        
def crawlingPage(name, rev_list):
    place = name
    review_list = rev_list
    idx = 3
    
    #첫 페이지
    extractReview(review_list)
    #mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(1) > a
    # 2~5 페이지
    try:
        page_num = len(driver.find_elements(by=By.CLASS_NAME, value='link_page')) # 페이지 수 찾기(기본 5개)
        for i in range(page_num-1): #기본 5개에서 현재 페이지 1개 제외
            driver.find_element(by=By.CSS_SELECTOR, value='#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
            time.sleep(2)
            extractReview(review_list)
            idx += 1
        driver.find_element(by=By.LINK_TEXT, value='다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
        time.sleep(2)
        extractReview(review_list) # 6번째 페이지 리뷰 추출
    except (NoSuchElementException, ElementNotInteractableException):
        print("no review in crawling")
        
    # 7페이지 부터
    while True:
        idx = 4
        try:
            page_num = len(driver.find_elements(by=By.CLASS_NAME, value='link_page')) #5
            for i in range(page_num-1): 
                driver.find_element(by=By.CSS_SELECTOR, value='#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                time.sleep(2)
                extractReview(review_list)
                idx += 1
            driver.find_element(by=By.LINK_TEXT, value='다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
            time.sleep(2)
            extractReview(review_list) # 리뷰 추출
        except:
            print("no review in crawling")
            break
    # 리뷰 텍스트 추출
    createText(place, review_list)
    

def createText(name, rev_list):
    restaurant_name = name
    review_list = rev_list
    with open(f"{restaurant_name}.txt",'w', encoding='utf-8') as f:
        for rev in review_list:
            f.write(rev+'\n')

In [4]:
for i in range(len(df)):
    print('======================================================')
    print(str(i)+'번째 식당')
    # 식당 리뷰 개별 url 접속
    driver.get(df['URL'][i])
    time.sleep(2)
    
    name = df['상호명'][i]
    rev_list = []
    
    crawlingPage(name, rev_list)
print('크롤링 완료')

0번째 식당
매번 포장해서 먹는데 냄새도 안 나고 맛있어요~~	10
웨이팅 심하지만 맛있어요	10
해장국 맛이 끝내줘요!!	10
맛집 인정!!	10
맛있어요~~	10
맛은 끝내준다!! 12000원이고 소주5천원 맥주6천원 사람 많아서 시끄럽고, 종업원이 좀 불친절하다는게 흠	10
맛, 양 다 좋지만 가격이 너무 올랐다~ 가격 땜시 3점.	10
가격 보소 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ에혀 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ	10
맛있다는 소문 듣고 일부러 찾아 갔는데 밥인지 떡인지 모르겠는 밥과 쏘쏘한 내장탕을 14000씩이나 주고 먹었네요 재방문 의사 많이 없음 	10
비싸다.기본 해장국 12천원. 그리고 짜다. 밥은 어제 했는지 떡밥줌. 서울근교 이름좀 난집 가면 9천원-1만원이면 충분. 재방문 의사 없음. 	10
별한개도 아까운 쓰레기 같은곳 직원인성교육부터 제대로 시켜야하는 곳 	10
여기 맛있다는 사람 진심으로 병원가서 미각 검사받아봐라. 깊고 진한 미원맛. 별한개도 아깝다. 맛 없다. 왜 유명한건가 이해 할 수 없었음.	10
가격대가 조금 있는편이지만 맛도 양도 만족스럽다	10
존맛 중앙해장. 저녁시간 웨이팅이 엄청나지만 애매한 시간에 가면 기다리지 않고 먹을 수 있음.	10
한국에서 제일 맛있는 내장탕, 곰창전골	10
여기 곱창전골이 맛있다고 느껴지면 당신의 미각은 빵점인걸로 할게요~	10
곱창전골은 입문자에게만 추천할게요.. 곱창전골 특유의 뭔가가 살짝 부족.. 곱창냄새가 하나도 안나긴 해요 곱도 빵빵 근데 기름이 많이붙어있슴..	10
곱창전골 먹으려 갖다  직원인지 메니저인지 불친절한 대응으로 나왔다. QR코드 입력하고 앉아 기다리는데 다른 직원이 또 QR 요구하여 했다니 믿을 수 없다며 계속요구 한다. 그럼 직원끼리 확인할 것이지 무례한 행동...  전골집이 여기뿐인가요? 	10
깊고 진한 맛이 좋아요~~	10
맛있고 양도 굿굿이예요!	10
저는 내장탕 먹었는데, 맛

그 가격에 그 맛	10
비싸긴 한데 맛은 확실히 있네요	10
곱창을 안좋아하는데도 여기 곱창전골이 인생맛집일 정도로 맛있어요.	10
너무 맛있음	10
곱창 전골이랑 중앙전골만 먹어봐서 두개 기준으로 평가함...  "내장으로 뽑은 육수+ 미리 양념한 곱창+다데기" 3콤보로 맛내는 집이 이제 서울시내 또 남았을까...   단점은 없고, 24시간 한다는 점에서 별5개 지만 아래 사유로 4개만 줌   단점 사유 : 주차 (근처 유료주차장 1시간에 만원나옴...),                    가성비나쁨 (...	10
웨이팅이 엄청나지만 맛있어요	10
명성듣고 찾아갔는데 실망은 안하더라~ 묽은 국물이 특징인데 밍밍하지 않음.	10
웨이팅이 꽤 길었는데도 먹자마자 약간 마음 풀리는맛... 진짜 괜찮은 곱창전골	10
24시에 테이블도 많은데 이렇게 웨이팅 많은 식당은 처음이에요 곱창전골 가격으로는 비싼데 맛있어요 우동사리를 줘서 곱창전골중짜리에 두명이 다른거 추가없이 배부르게 먹었습니다 채소는 중간에 오셔서 잘라주시던데 남자들만 있는 테이블은 중간중간 불도 봐주시고 여자끼리 있는 테이블은 요청해야 잘라주시네요 좀 아쉽습니다	10
존맛탱, 4.5 주고 싶었는데 .5가 없다 술 생각 없었는데, 국물 한 입 먹고 바로 시켰당 ㅋㅑ~~	10
Beef Soup 	10
포장은 안 된대요! 계속 생각나는 맛.	10
그냥 그럼	10
비싸지만 깔끔하고 맛있음	10
곱창전골에서 스티로폼 조각 나왔어요. 위생도 그렇지만 버섯이라며 먹어도 된다고 말하는 눈 가리고 아웅하는 듯한 대처도 상당히 별로였습니다. 결국 스티로폼인거 인정받고 환불 받았습니다.	10
최애메뉴는 곱창전골이입니다. 워낙 인기가 많은 곳이라 대기가 어마어마 하지만 ㅠㅠ 친절하고 맛있어요	10
제가 먹어본 해장국중 가장 맛있고, 고급진 해장국이에요.  부속품이 실해요.  진짜 먹고, 힐링되었음.	10
너무 맛있어요	10
사람이 많고 그닥 친절하진 않지만 맛은 진짜 최고 중앙전골 먹었는데 내장전골임 냄새 하나도 안나고 양념이